# S&P 500 Stock Analysis - 2025 H1

This notebook analyzes stock data from the S&P 500 for the first half of 2025.

**Dataset:** `sp500_2025_h1_wide_clean.csv`

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('Libraries loaded successfully!')

## 1. Load and Explore the Data

In [ ]:
# Load the dataset
df = pd.read_csv('sp500_2025_h1_wide_clean.csv')

# Display basic information
print(f'Dataset shape: {df.shape}')
print(f'Number of companies: {len(df)}')
print(f'Number of columns: {len(df.columns)}')
print('
First few rows:')
df.head()

In [ ]:
# Display column names
print('Column names:')
for i, col in enumerate(df.columns):
    print(f'{i}: {col}')

In [ ]:
# Data types and missing values
print('Data types:')
print(df.dtypes.value_counts())
print('
Missing values:')
print(df.isnull().sum().sum())

## 2. Extract Date Columns

In [ ]:
# Identify opening, closing, and volume columns
opening_cols = [col for col in df.columns if 'opening' in col.lower()]
closing_cols = [col for col in df.columns if 'closing' in col.lower()]
volume_cols = [col for col in df.columns if 'volume' in col.lower()]

print(f'Opening price columns: {len(opening_cols)}')
print(f'Closing price columns: {len(closing_cols)}')
print(f'Volume columns: {len(volume_cols)}')
print(f'
Total trading days: {len(opening_cols)}')

## 3. Top Performers Analysis

In [ ]:
# Calculate average closing price for each company
df['avg_closing_price'] = df[closing_cols].mean(axis=1)

# Top 10 companies by average closing price
top_10_by_price = df.nlargest(10, 'avg_closing_price')[['company_name', 'ticker', 'avg_closing_price']]
print('Top 10 Companies by Average Closing Price:')
print(top_10_by_price.to_string(index=False))

In [ ]:
# Calculate price change (first closing price vs last closing price)
df['first_close'] = df[closing_cols[0]]
df['last_close'] = df[closing_cols[-1]]
df['price_change'] = df['last_close'] - df['first_close']
df['price_change_pct'] = (df['price_change'] / df['first_close']) * 100

# Top 10 gainers
top_gainers = df.nlargest(10, 'price_change_pct')[['company_name', 'ticker', 'first_close', 'last_close', 'price_change_pct']]
print('
Top 10 Gainers (%):')
print(top_gainers.to_string(index=False))

In [ ]:
# Top 10 losers
top_losers = df.nsmallest(10, 'price_change_pct')[['company_name', 'ticker', 'first_close', 'last_close', 'price_change_pct']]
print('Top 10 Losers (%):')
print(top_losers.to_string(index=False))

## 4. Volume Analysis

In [ ]:
# Calculate average trading volume
df['avg_volume'] = df[volume_cols].mean(axis=1)

# Top 10 most traded stocks by volume
top_by_volume = df.nlargest(10, 'avg_volume')[['company_name', 'ticker', 'avg_volume']]
print('Top 10 Most Traded Stocks by Average Volume:')
print(top_by_volume.to_string(index=False))

## 5. Visualizations

In [ ]:
# Plot top 10 gainers
plt.figure(figsize=(12, 6))
top_gainers_plot = df.nlargest(10, 'price_change_pct')
plt.barh(top_gainers_plot['ticker'], top_gainers_plot['price_change_pct'], color='green')
plt.xlabel('Price Change (%)')
plt.ylabel('Company Ticker')
plt.title('Top 10 Gainers - H1 2025')
plt.tight_layout()
plt.show()

In [ ]:
# Plot top 10 losers
plt.figure(figsize=(12, 6))
top_losers_plot = df.nsmallest(10, 'price_change_pct')
plt.barh(top_losers_plot['ticker'], top_losers_plot['price_change_pct'], color='red')
plt.xlabel('Price Change (%)')
plt.ylabel('Company Ticker')
plt.title('Top 10 Losers - H1 2025')
plt.tight_layout()
plt.show()

In [ ]:
# Distribution of price changes
plt.figure(figsize=(12, 6))
plt.hist(df['price_change_pct'], bins=50, color='skyblue', edgecolor='black')
plt.xlabel('Price Change (%)')
plt.ylabel('Frequency')
plt.title('Distribution of Price Changes - H1 2025')
plt.axvline(df['price_change_pct'].mean(), color='red', linestyle='--', label=f'Mean: {df["price_change_pct"].mean():.2f}%')
plt.axvline(df['price_change_pct'].median(), color='green', linestyle='--', label=f'Median: {df["price_change_pct"].median():.2f}%')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Top 10 by average volume
plt.figure(figsize=(12, 6))
top_volume_plot = df.nlargest(10, 'avg_volume')
plt.barh(top_volume_plot['ticker'], top_volume_plot['avg_volume']/1e6, color='purple')
plt.xlabel('Average Volume (Millions)')
plt.ylabel('Company Ticker')
plt.title('Top 10 Most Traded Stocks by Volume - H1 2025')
plt.tight_layout()
plt.show()

## 6. Statistical Summary

In [ ]:
# Overall market statistics
print('=== Market Statistics - H1 2025 ===')
print(f'Average price change: {df["price_change_pct"].mean():.2f}%')
print(f'Median price change: {df["price_change_pct"].median():.2f}%')
print(f'Standard deviation: {df["price_change_pct"].std():.2f}%')
print(f'
Number of gainers: {(df["price_change_pct"] > 0).sum()}')
print(f'Number of losers: {(df["price_change_pct"] < 0).sum()}')
print(f'Number of unchanged: {(df["price_change_pct"] == 0).sum()}')
print(f'
Percentage of gainers: {(df["price_change_pct"] > 0).sum() / len(df) * 100:.2f}%')
print(f'Percentage of losers: {(df["price_change_pct"] < 0).sum() / len(df) * 100:.2f}%')

## 7. Price Range Analysis

In [ ]:
# Calculate min and max prices for each stock
df['min_price'] = df[closing_cols].min(axis=1)
df['max_price'] = df[closing_cols].max(axis=1)
df['price_range'] = df['max_price'] - df['min_price']
df['volatility_pct'] = (df['price_range'] / df['min_price']) * 100

# Most volatile stocks
most_volatile = df.nlargest(10, 'volatility_pct')[['company_name', 'ticker', 'min_price', 'max_price', 'volatility_pct']]
print('Top 10 Most Volatile Stocks:')
print(most_volatile.to_string(index=False))

In [ ]:
# Least volatile stocks
least_volatile = df.nsmallest(10, 'volatility_pct')[['company_name', 'ticker', 'min_price', 'max_price', 'volatility_pct']]
print('Top 10 Least Volatile Stocks:')
print(least_volatile.to_string(index=False))

## 8. Specific Stock Analysis (Example: Top 5 Tech Stocks)

In [ ]:
# Select some tech stocks for detailed analysis
tech_stocks = ['NVDA', 'MSFT', 'AAPL', 'AMZN', 'META']
tech_df = df[df['ticker'].isin(tech_stocks)]

# Plot their price movements
plt.figure(figsize=(14, 7))
for idx, row in tech_df.iterrows():
    prices = row[closing_cols].values
    plt.plot(range(len(prices)), prices, marker='o', label=row['ticker'], linewidth=2)

plt.xlabel('Trading Day')
plt.ylabel('Closing Price ($)')
plt.title('Price Movement - Top 5 Tech Stocks (H1 2025)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Export Summary Report

In [ ]:
# Create a summary dataframe
summary_df = df[['company_name', 'ticker', 'first_close', 'last_close', 'price_change', 'price_change_pct', 'avg_volume', 'volatility_pct']].copy()
summary_df = summary_df.sort_values('price_change_pct', ascending=False)

# Save to CSV
summary_df.to_csv('sp500_analysis_summary.csv', index=False)
print('Summary report saved to sp500_analysis_summary.csv')
summary_df.head(20)

## 10. Conclusions

This notebook provides a comprehensive analysis of S&P 500 stocks for H1 2025, including:

- **Performance Analysis**: Identified top gainers and losers
- **Volume Analysis**: Found the most actively traded stocks
- **Volatility Analysis**: Determined which stocks had the most price movement
- **Market Overview**: Calculated overall market statistics
- **Visual Insights**: Created charts for better understanding of trends

You can modify the analysis by changing parameters, adding more visualizations, or focusing on specific sectors or stocks.